# Level 2: Simple Agentic 

This tutorial is designed to give new users an overview of how to use llama-stack's builtin agent framework effectively. It demonstrates basic single-tool usage and provides guidance on switching between a local development environment and a remote kubernetes cluster.

## Overview

This tutorial walks you through how to build your own AI agent who can search the web following these steps:
1. Connecting to a llama-stack server (remote or local).
2. Configuring an agent for tool use.
3. Running the agent.


## Prerequisites

Before starting, ensure you have the following:
- Access to a remote cluster or a local Podman setup.
- A Tavily API key is required. You can register for one at [https://tavily.com/](https://tavily.com/).

### Setting your ENV variables:

Use the [`.env.example`](../../../.env.example) to create a new file called `.env` and ensure you add all the relevant environment variables below. 

- `REMOTE_BASE_URL` (string): the URL for your llama-stack instance if using remote connection.
- `TAVILY_SEARCH_API_KEY` (string): your API key for tavily search. One can get one by going to: https://tavily.com/home.

### Installing dependencies

This code requires `llama-stack` and the `llama-stack-client`, both at version `0.2.2`. Lets begin by installing them:


In [ ]:
# !pip install llama-stack-client==0.2.2 llama-stack==0.2.2

## Building a Web Search Agent
### 1. Connecting to a llama-stack server.
#### 1.1 Setting Up the Environment
- Import the necessary libraries.
- Defining user variables.
- Connection options
    - **Remote Server**: Set `remote=True` and provide the remote llama-stack endpoint.
    - **Local Server**: Set `remote=False` and provide the local llama-stack endpoint (default port: 8321).
    
    For detailed llama-stack server setup instructions, refer to:
    - [Remote Setup Guide](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/kubernetes)
    - [Local Setup Guide](https://github.com/opendatahub-io/llama-stack-demos/blob/main/local_setup_guide.md)

In [5]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client import LlamaStackClient
import os
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint
from dotenv import load_dotenv
load_dotenv()

remote = True # Use the `remote` variable to switching between a local development environment and a remote kubernetes cluster.
stream_output = False # Set to True to stream the output of the agent.
inference_model = "ibm-granite/granite-3.2-8b-instruct"
print(f"Model: {inference_model}")

tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")  # necessary for websearch tool
if remote:
    base_url = os.getenv("REMOTE_BASE_URL")
else:
    base_url = "http://localhost:8321"

Model: ibm-granite/granite-3.2-8b-instruct


#### 1.2 Client Initialization
- Initialize the `LlamaStackClient` with the appropriate base URL and provider data.  
- Connect the llama-stack client to the llama-stack server using the base URL.

In [6]:
client = LlamaStackClient(
        base_url=base_url,
        provider_data={"tavily_search_api_key": tavily_search_api_key}
)

### 2. Configuring an agent for tool use.
- **Agent Initialization**: 

- Create an `Agent` instance with the desired model, instructions and tools."

- **Instructions**: The `instructions` parameter, also referred to as the system prompt, specifies the agent's role and behavior. In this example, the agent is configured as a helpful web search assistant. It is instructed to use a tool whenever a web search is required and to respond in a friendly and helpful tone.

- **Tools**: The `tools` parameter defines the tools available to the agent. In this case, the `builtin::websearch` tool is used, which enables the agent to perform web searches. This tool is essential for retrieving up-to-date information from the web. Internally, it leverages Tavily Search to execute the search queries efficiently.

- **How It Works**: When a user query is provided, the agent processes the input and determines whether a tool is required to fulfill the request. If the query involves retrieving information from the web, the agent invokes the `builtin::websearch` tool. The tool interacts with Tavily Search to fetch real-time data, which is then processed and returned to the user in a friendly and helpful tone. This workflow ensures that the agent can handle a wide range of queries effectively.

For more details on the `builtin::websearch` tool and its capabilities, refer to the [Llama-stack tools documentation](https://llama-stack.readthedocs.io/en/latest/building_applications/tools.html#web-search-providers). 

In [16]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params={"max_tokens":4096}
)

### 3. Running the agent.
- Define user prompts to interact with the agent.
- Use the agent to create a session and process user queries.
- Display the agent's responses for each query.

In [17]:
user_prompts = [
    "What’s latest in OpenShift?",
    "Search for recent news articles about advancements in quantum computing.",
    "Find coffee shops near Dublin city centre that are open now, have Wi-Fi.",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: What’s latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'latest updates on OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'latest updates on OpenShift',
│   'top_k': [
│   │   {
│   │   │   'title': 'Azure Red Hat OpenShift: April 2025 Update | Microsoft Community Hub',
│   │   │   'url': 'https://techcommunity.microsoft.com/blog/appsonazureblog/azure-red-hat-openshift-april-2025-update/4400331',
│   │   │   'content': "Azure Red Hat OpenShift's April update boosts security with Managed Identity, expands global reach, and enhances the platform for enterprise Kubernetes. ... OpenShift 4.16 on Day 0 → Now available for new ARO cluster installs with the latest features and improvements.",
│   │   │   'score': 0.8377398,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat unveils OpenShift 4.18: Enhanced security and virtualization ...',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-418',
│   │   │   'content': 'OLM also enables complementary solutions such as\xa0 Red Hat Advanced Cluster Management for Kubernetes,\xa0Red Hat Advanced Cluster Security for Kubernetes,\xa0Red Hat OpenShift Service Mesh and Red Hat OpenShift Virtualization, while unlocking advanced workloads like\xa0GPU,\xa0Precision Time Protocol (PTP) and\xa0SR-IOV networking Building on these achievements, OLM is advancing toward more streamlined APIs and enhanced declarative integration with\xa0Red Hat OpenShift GitOps. Key OLM enhancements include: The latest release introduces\xa0cert-manager Operator for Red Hat OpenShift v1.15 with technology preview of the powerful\xa0istio-csr agent, enabling seamless integration to secure\xa0Red Hat OpenShift Service Mesh workload and control plane components. Nick Png is a Technical Product Manager for the Red Hat OpenShift Container Platform, focusing primarily on security related products that enable customers to securely run workloads on an OpenShift cluster.',
│   │   │   'score': 0.72698855,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift 4.17: What you need to know',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-417',
│   │   │   'content': 'Based on\xa0Kubernetes 1.30 and\xa0CRI-O 1.30,\xa0 OpenShift 4.17 features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in\xa0Red Hat OpenShift Platform Plus. Red Hat OpenShift Platform Plus (OPP) is an expanded version of Red Hat OpenShift that provides a comprehensive hybrid cloud platform with built-in security features for enterprises to build, deploy, run, manage, and automate applications at scale. For\xa0Red Hat OpenShift Data Foundation (ODF), we’re promoting to general availability the use of customer-managed ODF with\xa0Red Hat OpenShift Service on AWS with hosted control planes clusters in ODF 4.17.z. Ju Lim works on the core Red Hat OpenShift Container Platform for hybrid and multi-cloud environments to enable customers to run Red Hat OpenShift anywhere.',
│   │   │   'score': 0.6633424,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift Container Platform Life Cycle Policy',
│   │   │   'url': 'https://access.redhat.com/support/policy/updates/openshift/',
│   │   │   'content': 'Extended Update Support - Long Life Add-on - Additional Term 1 enables customers to remain with the same minor release of Red Hat OpenShift for a total 24 months, allowing for stable production environments for mission-critical applications. Extended Update Support - Long Life Add-on - Additional Term 1 is provided with all Premium subscriptions for x86-64 versions of Red Hat OpenShift Kubernetes Engine, Red Hat OpenShift Container Platform, and Red Hat OpenShift Platform Plus. The combination of Extended Update Support - Long Life Add-On - Additional Term 1 and Extended Update Support Long Life Add-on - Additional Term 2 (footnote [13]) enables customers to remain with the same mi


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are the latest updates on OpenShift:

1. **Azure Red Hat OpenShift: April 2025 Update**
   - Boosts security with Managed Identity.
   - Expands global reach.
   - Enhances the platform for enterprise Kubernetes.
   - OpenShift 4.16 is now available for new ARO cluster installs with the latest features and improvements.

2. **Red Hat OpenShift 4.18: Enhanced Security and Virtualization**
   - OLM (Operator Lifecycle Manager) enables complementary solutions like Red Hat Advanced Cluster Management, Red Hat Advanced Cluster Security, Red Hat OpenShift Service Mesh, and Red Hat OpenShift Virtualization.
   - The latest release introduces cert-manager Operator for Red Hat OpenShift v1.15 with a technology preview of the istio-csr agent for seamless integration with Red Hat OpenShift Service Mesh.

3. **Red Hat OpenShift 4.17: What you need to know**
   - Based on Kubernetes 1.30 and CRI-O 1.30.
   - Features expanded con

{
│   'query': 'recent news articles about advancements in quantum computing',
│   'top_k': [
│   │   {
│   │   │   'title': 'Quantum Computing News -- ScienceDaily',
│   │   │   'url': 'https://www.sciencedaily.com/news/matter_energy/quantum_computing/',
│   │   │   'content': "Quantum Computing News March 2, 2025 Top Headlines Quantum Interference in Molecule-Surface Collisions Feb. 27, 2025 — Scientists have revealed how quantum interference and symmetry dictate molecular behavior in collisions with gold surfaces, offering new insights into molecular interactions. Unraveling How a 'Magnetic Twist' Induces One-Way Electric Flow Feb. 27, 2025 — A twist you'll never see coming: a breakthrough in understanding the relationship between chirality and electric flow at a microscopic level may help us develop chiral information ... Materials Incorporated Into Quantum Qubit Platform Feb. 25, 2025 — Researchers detail advances in the measurement of quantum devices that will be needed to realize a topological quantum computer. The ... New Low-Cost Challenger to Quantum Computer: Ising Machine Feb. 26, 2025 — A low-energy challenger to the quantum computer also works at room temperature. Quantum sensors ... Topological Quantum Processor Marks Breakthrough in Computing Feb. 20, 2025 — In a leap forward for quantum computing, physicists unveiled an eight-qubit topological quantum processor, the first of its kind.",
│   │   │   'score': 0.8815438,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Recent Breakthroughs Accelerate The Race For Quantum Computing - Forbes',
│   │   │   'url': 'https://www.forbes.com/sites/drektadang/2025/03/09/recent-breakthroughs-accelerate-the-race-for-quantum-computing/',
│   │   │   'content': "The race toward scalable quantum computing has reached a pivotal moment, with major players like Microsoft, Google, and IBM pushing forward with breakthroughs. Microsoft's recent announcement of its Majorana 1 chip marks a significant milestone, while Google’s Willow chip and IBM’s long-term quantum roadmap illustrate the industry’s diverse approaches to achieving fault-tolerant quantum systems. Unlike conventional qubit technologies, which require extensive error correction, Microsoft’s approach aims to build fault tolerance directly into the hardware, significantly improving the feasibility of large-scale quantum computing. While Microsoft, Google, and IBM push forward, Nvidia CEO Jensen Huang remains skeptical, stating at CES 2025 that quantum computing’s practical use may still be 20 years away.",
│   │   │   'score': 0.86331123,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Quantum computing | MIT News | Massachusetts Institute of Technology',
│   │   │   'url': 'https://news.mit.edu/topic/quantum-computing',
│   │   │   'content': 'February 5, 2025 Read full story → Fast control methods enable record-setting fidelity in superconducting qubit The advance holds the promise to reduce error-correction resource overhead. January 13, 2025 Read full story → MIT physicists predict exotic form of matter with potential for quantum computing New work suggests the ability to create fractionalized electrons known as non-Abelian anyons without a magnetic field, opening new possibilities for basic research and future applications. October 30, 2024 Read full story → Toward a code-breaking quantum computer Building on a landmark algorithm, researchers propose a way to make a smaller and more noise-tolerant quantum factoring circuit for cryptography. July 31, 2024 Read full story → Modular, scalable hardware architecture for a quantum computer A new quantum-system-on-chip enables the efficient control of a large array of qubits, moving toward practical quantum computing. May 2, 2024 Read full story → MIT scientists tune the entanglement structure in an array of qubits The advance offers a way to characterize a fundamental resource needed for quantum computing.',
│   │   │   '


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some recent news articles about advancements in quantum computing:

1. **Quantum Computing News -- ScienceDaily**
   - Title: "Quantum Computing News March 2, 2025 Top Headlines"
   - Content: This article covers several recent breakthroughs in quantum computing, including quantum interference in molecule-surface collisions, understanding the relationship between chirality and electric flow, and the development of a topological quantum processor.
   - [Read Full Story](https://www.sciencedaily.com/news/matter_energy/quantum_computing/)

2. **Recent Breakthroughs Accelerate The Race For Quantum Computing - Forbes**
   - Title: "Recent Breakthroughs Accelerate The Race For Quantum Computing"
   - Content: This piece discusses the rapid progress in the field, with major companies like Microsoft, Google, and IBM making significant strides. It highlights Microsoft's Majorana 1 chip, Google's Willow chip, and IBM's lon

{
│   'query': 'coffee shops near Dublin city centre open now with Wi-Fi',
│   'top_k': [
│   │   {
│   │   │   'title': '10 Best Places To Grab A Coffee And Work In Dublin With Free Wi-Fi',
│   │   │   'url': 'https://lovin.ie/travel-1/cities/10-best-places-to-grab-a-coffee-and-work-in-dublin-with-free-wi-fi',
│   │   │   'content': '10 Best Places To Grab A Coffee And Work In Dublin With Free Wi-Fi - Lovin.ie 10 Best Places To Grab A Coffee And Work In Dublin With Free Wi-Fi BOI are trying to give something back with their slick hub in Grand Canal Square with comfy seating and free Wi-Fi. You’ll have to grab your own coffee from any outlet close-by and be comfortable with swapping your personal details for free Wi-Fi but you might as well get something free from the bank without breaking the bank! If you’re in the market for some incredible tasty salads, 3FE coffee, free Wi-Fi, like being surrounded by creatives and possibly have your dog in tow then The Fumbally Cafe is your spot.',
│   │   │   'score': 0.81204927,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Best Free Wi-Fi Cafes in Dublin - UPDATED: November, 2024',
│   │   │   'url': 'https://laptopfriendlycafe.com/free-wifi-cafes-in-dublin',
│   │   │   'content': "For those craving a cozy spot to work or study while enjoying a cup of coffee, Dublin boasts a plethora of cafes with free wifi that cater to students, freelancers, and remote workers. Many cafes in Dublin not only offer free wifi, but also serve delicious pastries, sandwiches, and specialty coffees to keep you fueled and focused throughout the day. So grab your laptop, order a hot cup of coffee, and settle in for a productive day of work or study at one of Dublin's best cafes with free wifi. Experience the charming and bustling atmosphere of our cafe, perfect for a delightful meal with friends or a quick coffee break. This café is a delightful hidden gem, perfect for those seeking a cozy atmosphere combined with great coffee and fresh food.",
│   │   │   'score': 0.76554126,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '10 Best Places To Grab A Coffee And Work In Dublin With Free Wifi',
│   │   │   'url': 'https://lovindublin.com/dublin/best-places-work-remotely-dublin-free-wifi',
│   │   │   'content': "In compiling this list, I've made sure each places met the vitally important criteria: a good strong free wifi connectio, plugs to charge laptops or phone, and of course, good coffee. All of the venues selected below tick all the boxes and are spread out around the city, to give you an option for all situations.",
│   │   │   'score': 0.6227582,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "11 Brilliant Cafés And Coffee Shops in Dublin: A Local's Guide",
│   │   │   'url': 'https://www.timeout.com/dublin/restaurants/best-cafes-in-dublin',
│   │   │   'content': 'Dublin has chic coffee shops in spades (in fact it has so many great spots, choosing the best is not an easy task). For coffee, sweet treats and everything in between, here are the best cafés in Dublin. The best coffee shops in Dublin Brindle Coffee & Wine is one of the best coffee shops in the city for sitting down in the sunshine and watching the world go by along the terraced street. Dublin’s top contemporary menswear shop is also the best place in Temple Bar to grab a coffee. Serving up baked goods from No Messin’ Bakery, Proper Order is a firm favourite with Dublin’s coffee lovers. This local landmark in the cosy neighbourhood of Ranelagh is one of Dublin’s most beloved coffee shops.',
│   │   │   'score': 0.5440511,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '17 Best Coffee Shops in Dublin - The Irish Road Trip',
│   │   │   'url': 'https://www.theirishroadtrip.com/best-coffee-shops-dublin/',
│   │   │   'content': "There are some mighty coffee shops in Dublin. In this guide, you'll find our favourites, split into cafes that ar


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some coffee shops near Dublin city centre that are open now and offer Wi-Fi:

1. **BOI Hub, Grand Canal Square**: This spot by the bank offers free Wi-Fi and comfortable seating. You'll need to grab your coffee from a nearby outlet. It's a great place for a productive work session. [More info](https://lovin.ie/travel-1/cities/10-best-places-to-grab-a-coffee-and-work-in-dublin-with-free-wi-fi)

2. **The Fumbally Cafe**: Known for its incredible salads, 3FE coffee, and free Wi-Fi, this cafe is perfect for those who want to work or study while enjoying their coffee. They're also dog-friendly. [More info](https://lovindublin.com/dublin/best-places-work-remotely-dublin-free-wifi)

3. **Various Cafes in Dublin**: Dublin is filled with cafes that offer free Wi-Fi, good coffee, and sometimes even plugs for charging your devices. Some popular ones include Brindle Coffee & Wine, Proper Order, and many more. [More info](htt

## Output Analysis
Here, we can observe that the `builtin::websearch` tool is used to perform a web search. The outputs are displayed in the notebook with color-coded text to help interpret the process:

- **Blue Text**: Represents the user's input or query.
- **Yellow Text**: Displays the LLM's inference response. 
- **Green Text**: Indicates the tool execution process, such as the tool being called and the query being sent to the web search API.

Great! 
We can see that the model returned some relevant and recent information about OpenShift. This was only possible due to its ability to call tools like the web search, demonstrating the agent's capacity to retrieve real-time data effectively.

For the second query, the model fetched the latest news articles, including 2 from March 2025. This is particularly impressive since the Granite 3.2 8B model, released in February 2025, has a knowledge cutoff date of April 2024. This demonstrates the agent's capability to extend beyond its static knowledge base by dynamically retrieving current information.  

For the third query about nearby coffee shops, the agent effectively used the `builtin::websearch` tool to provide practical, location-specific results with valid coffee shops and links, highlighting its real-world applicability.  

## Testing location-based queries


### Basic web search tool agent in one shot

In [18]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params={"max_tokens":4096}
)
user_prompts = [
    "Find coffee shops near me that are open now, have Wi-Fi.",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Find coffee shops near me that are open now, have Wi-Fi.

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'coffee shops open now with Wi-Fi near me'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'coffee shops open now with Wi-Fi near me',
│   'top_k': [
│   │   {
│   │   │   'title': 'THE BEST 10 Coffee & Tea in DEARBORN, MI - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?cflt=coffee&find_loc=Dearborn,+MI',
│   │   │   'content': 'Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor Seating. 1. ... Coffee Shops Open Late in Dearborn, Michigan. Vegan Breakfast in Dearborn, Michigan. Cozy Coffee Shop With Couches in Dearborn, Michigan. Pastries and Coffee in Dearborn, Michigan. Breakfast Burrito in Dearborn, Michigan.',
│   │   │   'score': 0.5117904,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Locations and Hours - Haraz Coffee House',
│   │   │   'url': 'https://haraz-coffee.myshopify.com/pages/locations-and-hours',
│   │   │   'content': 'come & visit our coffee Shops. Dearborn. 13810 Michigan Ave, Dearborn, MI 48126, US Open now • Closes at 11PM All hours. View Menu Order Online. East Lansing. 501 E Grand River Ave, East Lansing, MI 48823, US ... Open now • Closes at 11PM All hours. View Menu Order Online',
│   │   │   'score': 0.2834619,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Qahwah - Qahwah House',
│   │   │   'url': 'https://qahwahhouse.com/',
│   │   │   'content': 'Qahwah House is an absolute gem in the world of coffee shops. From the moment I walked in, I was greeted with a warm and inviting atmosphere. The cozy decor, coupled with the aroma of freshly brewed coffee, immediately put me at ease. The coffee here is simply outstanding. They take great care in sourcing their beans and it shows in every sip.',
│   │   │   'score': 0.12716432,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '17 Best Coffee Shops in Dearborn, MI - CoffeeSpots.com',
│   │   │   'url': 'https://coffeespots.com/best-coffee-shops-in-dearborn-mi/',
│   │   │   'content': 'Biggby Coffee (Halal) 9.2/10 (578 reviews). Website | Directions | +1 313-982-1100. Great Coffee Friendly Team Clean Atmosphere. Biggby Coffee (Halal) seems to have quite a loyal following, with customers praising the great taste and quality of the coffee, the friendly and helpful new management and team, and the clean and inviting atmosphere of the coffee shop.',
│   │   │   'score': 0.118273266,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Home - City Coffeehouse',
│   │   │   'url': 'https://citycoffeehouseofficial.com/',
│   │   │   'content': "An independent coffee shop downriver earns an extra star no matter what. My idea of a great coffee shop is one that is spacious, friendly, quiet and has excellent coffee, tea and pastries. It's also always a plus when a coffee shop embraces the local community ie art, bulletins, events, etc. While there is arguably better coffee out there.",
│   │   │   'score': 0.107315995,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some coffee shops near you that are currently open and offer Wi-Fi:

1. **THE BEST 10 Coffee & Tea in DEARBORN, MI - Yelp**
   - They are open now and offer delivery, takeout, free Wi-Fi, and outdoor seating. They also have vegan breakfast options, cozy couches, and pastries.
   - [Check them out on Yelp](https://www.yelp.com/search?cflt=coffee&find_loc=Dearborn,+MI)

2. **Haraz Coffee House**
   - Their Dearborn location is open now and closes at 11PM. They also offer delivery and takeout.
   - [View their menu and order online](https://haraz-coffee.myshopify.com/pages/locations-and-hours)

3. **Qahwah - Qahwah House**
   - This coffee shop is highly rated for its cozy atmosphere, freshly brewed coffee, and careful bean sourcing.
   - [Learn more about Qahwah House](https://qahwahhouse.com/)

4. **Biggby Coffee (Halal) - CoffeeSpots.com**
   - Known for great taste, quality, and a friendly team. They have a clea

### Web search agent + Prompt chaining 

In [19]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params={"max_tokens":4096}
)
user_prompts = [
    "Where am i?",
    "Im in Dublin, Ireland",
    "Find coffee shops near me that are open now, have Wi-Fi."
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am i?

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
I'm sorry for any confusion, but as a web search assistant, I don't have the ability to determine your physical location. I can help you find information about your location if you provide me with a city, country, or landmark.

========== Query processing completed ========== 


Processing user query: Im in Dublin, Ireland

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
Sure, I can help with that! Here are some current happenings in Dublin, Ireland:

1. **Dublin Pride**: This year's Dublin Pride Parade is scheduled for September 24th. It's a celebration of the LGBTQ+ community and a great event to attend if you're in town.

2. **Trinity College Dublin Open Days**: If you're interested in academia, Trinity College is hosting open days for prospective students. These events usually occur in September and October.

3. **Dublin Theatre Festival**: This annual event showcase

{
│   'query': 'coffee shops open now with Wi-Fi near me',
│   'top_k': [
│   │   {
│   │   │   'title': 'THE BEST 10 Coffee & Tea in DEARBORN, MI - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?cflt=coffee&find_loc=Dearborn,+MI',
│   │   │   'content': 'Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor Seating. 1. ... Coffee Shops Open Late in Dearborn, Michigan. Vegan Breakfast in Dearborn, Michigan. Cozy Coffee Shop With Couches in Dearborn, Michigan. Pastries and Coffee in Dearborn, Michigan. Breakfast Burrito in Dearborn, Michigan.',
│   │   │   'score': 0.5117904,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Locations and Hours - Haraz Coffee House',
│   │   │   'url': 'https://haraz-coffee.myshopify.com/pages/locations-and-hours',
│   │   │   'content': 'come & visit our coffee Shops. Dearborn. 13810 Michigan Ave, Dearborn, MI 48126, US Open now • Closes at 11PM All hours. View Menu Order Online. East Lansing. 501 E Grand River Ave, East Lansing, MI 48823, US ... Open now • Closes at 11PM All hours. View Menu Order Online',
│   │   │   'score': 0.2834619,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Qahwah - Qahwah House',
│   │   │   'url': 'https://qahwahhouse.com/',
│   │   │   'content': 'Qahwah House is an absolute gem in the world of coffee shops. From the moment I walked in, I was greeted with a warm and inviting atmosphere. The cozy decor, coupled with the aroma of freshly brewed coffee, immediately put me at ease. The coffee here is simply outstanding. They take great care in sourcing their beans and it shows in every sip.',
│   │   │   'score': 0.12716432,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '17 Best Coffee Shops in Dearborn, MI - CoffeeSpots.com',
│   │   │   'url': 'https://coffeespots.com/best-coffee-shops-in-dearborn-mi/',
│   │   │   'content': 'Biggby Coffee (Halal) 9.2/10 (578 reviews). Website | Directions | +1 313-982-1100. Great Coffee Friendly Team Clean Atmosphere. Biggby Coffee (Halal) seems to have quite a loyal following, with customers praising the great taste and quality of the coffee, the friendly and helpful new management and team, and the clean and inviting atmosphere of the coffee shop.',
│   │   │   'score': 0.118273266,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Home - City Coffeehouse',
│   │   │   'url': 'https://citycoffeehouseofficial.com/',
│   │   │   'content': "An independent coffee shop downriver earns an extra star no matter what. My idea of a great coffee shop is one that is spacious, friendly, quiet and has excellent coffee, tea and pastries. It's also always a plus when a coffee shop embraces the local community ie art, bulletins, events, etc. While there is arguably better coffee out there.",
│   │   │   'score': 0.107315995,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some coffee shops near you that are currently open and offer Wi-Fi:

1. **THE BEST 10 Coffee & Tea in DEARBORN, MI - Yelp**
   - They are open now and offer delivery, takeout, free Wi-Fi, and outdoor seating. They also have vegan breakfast options, cozy couches, and pastries.
   - [Check them out on Yelp](https://www.yelp.com/search?cflt=coffee&find_loc=Dearborn,+MI)

2. **Haraz Coffee House**
   - Their Dearborn location is open now and closes at 11PM. They also offer delivery and takeout.
   - [View their menu and order online](https://haraz-coffee.myshopify.com/pages/locations-and-hours)

3. **Qahwah - Qahwah House**
   - This coffee shop is highly rated for its cozy atmosphere, freshly brewed coffee, and careful bean sourcing.
   - [Learn more about Qahwah House](https://qahwahhouse.com/)

4. **Biggby Coffee (Halal) - CoffeeSpots.com**
   - Known for great taste, quality, and a friendly team. They have a clea

### Web search ReAct Agent

In [ ]:
agent = ReActAgent(
            client=client,
            model=inference_model,
            instructions="""You are a helpful websearch assistant. 
            When you are asked to search the web you must use a tool. 
            You are a helpful assistant that uses reasoning to solve problems step by step. Break down complex problems into simpler steps.
            """,
            tools=["builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params={"max_tokens":4096},
        )
user_prompts = [
    "Find coffee shops near me that are open now, have Wi-Fi."
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Find coffee shops near me that are open now, have Wi-Fi.

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
  "thought": "To find coffee shops near the user that are open now and have Wi-Fi, I will use the 'brave_search' tool. I will formulate a search query that includes the user's location, the requirement for the shops to be open now, and the need for Wi-Fi. I will then execute the search and present the results to the user.",
  "action": {
    "tool_name": "brave_search",
    "tool_params": [
      {
        "name": "query",
        "value": "coffee shops open now with Wi-Fi near me"
      }
    ]
  }

,
  "answer": "I'm currently searching for coffee shops that are open now and offer Wi-Fi near your location. Please wait a moment while I gather this information."
}

========== Query processing completed ========== 



## Key Takeaways
- This tutorial demonstrates how to set up and use a single tool with AI Agent.
- It highlights the flexibility of switching between remote and local environments.
- By following these steps, users can quickly get started with AI Agents and explore its capabilities.

For more advanced use cases, please check our other example jupyter notebooks.